# Fußball als Netzwerk daten analysieren

- Fußball ist eines der am besten getrackten Spiele überhaupt. Siehe https://statsbomb.com/statsbombiq/
- Ich habe einen kleinen freien Datensatz gefunden https://github.com/statsbomb/open-data
- Auch interessant https://spielverlagerung.com/bundesliga/

In [1]:
import json
import networkx as nx
import re

## Daten importieren

In [2]:
with open('19714.json') as f:
    data = json.load(f)

## Wie sehen die Daten eigentlich aus?

In [5]:
data[123]

{u'duration': 1.752473,
 u'id': u'c2676078-a261-4f1a-bb66-bc8c3b71b310',
 u'index': 124,
 u'location': [10.0, 57.0],
 u'minute': 3,
 u'off_camera': False,
 u'pass': {u'angle': 0.16514868,
  u'body_part': {u'id': 40, u'name': u'Right Foot'},
  u'end_location': [34.0, 61.0],
  u'height': {u'id': 1, u'name': u'Ground Pass'},
  u'length': 24.33105,
  u'outcome': {u'id': 9, u'name': u'Incomplete'}},
 u'period': 1,
 u'play_pattern': {u'id': 4, u'name': u'From Throw In'},
 u'player': {u'id': 17524, u'name': u'Jennifer Beattie'},
 u'position': {u'id': 5, u'name': u'Left Center Back'},
 u'possession': 7,
 u'possession_team': {u'id': 746, u'name': u'Manchester City WFC'},
 u'related_events': [u'1f523ab1-f91c-4bda-be6c-c2bfa431027a'],
 u'second': 17,
 u'team': {u'id': 746, u'name': u'Manchester City WFC'},
 u'timestamp': u'00:03:17.166',
 u'type': {u'id': 30, u'name': u'Pass'}}

# Spieler  
- Sie sind unsere Knoten

### Spieler Team 1

In [3]:
players1 = {}
for player in data[0]["tactics"]["lineup"]:
    players1[player["player"]["name"]] = player["position"]["name"]
players1["Drew Spence"] = "Einwechsel"
players1['Francesca Kirby'] = "Einwechsel"
players1["Erin Cuthbert"] = "Einwechsel"
players1

{u'Bethany England': u'Right Wing',
 'Drew Spence': 'Einwechsel',
 'Erin Cuthbert': 'Einwechsel',
 'Francesca Kirby': 'Einwechsel',
 u'Jessica Carter': u'Right Back',
 u'Jonna Andersson': u'Left Back',
 u'Karen Julia Carney': u'Left Wing',
 u'Magdalena Ericsson': u'Left Center Back',
 u'Maren N\xe6vdal Mjelde': u'Left Defensive Midfield',
 u'Millie Bright': u'Right Center Back',
 u'Ramona Bachmann': u'Center Forward',
 u'Rut Hedvig Lindahl': u'Goalkeeper',
 u'So-yun Ji': u'Center Attacking Midfield',
 u'Sophie Ingle': u'Right Defensive Midfield'}

### Spieler Team 2

In [4]:
players2 = {}
for player in data[1]["tactics"]["lineup"]:
    players2[player["player"]["name"]] = player["position"]["name"]
players2['Nadia Nadim']  = "Einwechsel"
players2['Janine Beckie'] = "Einwechsel"
players2

{u'Abbie McManus': u'Left Back',
 u'Gemma Bonner': u'Right Back',
 u'Georgia Stanway': u'Left Center Midfield',
 'Janine Beckie': 'Einwechsel',
 u'Jennifer Beattie': u'Left Center Back',
 u'Jill Louise Scott': u'Right Center Midfield',
 u'Karen Louise Bardsley': u'Goalkeeper',
 u'Keira Walsh': u'Center Midfield',
 u'Lauren Hemp': u'Right Center Forward',
 u'Melissa Lawley': u'Left Center Forward',
 'Nadia Nadim': 'Einwechsel',
 u'Nikita Parris': u'Center Forward',
 u'Stephanie Houghton': u'Right Center Back'}

## Einwechselspieler

In [10]:
list(set(players)-set(players1)-set(players2))

[]

## Pässe
- Sie sind unsere Kanten

In [6]:
out = []
for r in data:
    if "pass" in r.keys():
        out.append(r)

In [7]:
len(out)

947

In [8]:
out[123]

{u'duration': 2.184795,
 u'id': u'b26f20e7-67ee-4f4f-8f4f-99741fd63039',
 u'index': 362,
 u'location': [24.0, 21.0],
 u'minute': 11,
 u'off_camera': False,
 u'pass': {u'angle': 2.329174,
  u'body_part': {u'id': 38, u'name': u'Left Foot'},
  u'end_location': [6.0, 40.0],
  u'height': {u'id': 1, u'name': u'Ground Pass'},
  u'length': 26.172504,
  u'recipient': {u'id': 4640, u'name': u'Rut Hedvig Lindahl'}},
 u'period': 1,
 u'play_pattern': {u'id': 4, u'name': u'From Throw In'},
 u'player': {u'id': 4633, u'name': u'Magdalena Ericsson'},
 u'position': {u'id': 5, u'name': u'Left Center Back'},
 u'possession': 23,
 u'possession_team': {u'id': 971, u'name': u'Chelsea LFC'},
 u'related_events': [u'6b4d4fed-a68b-4821-bec4-d5c273c740b1'],
 u'second': 35,
 u'team': {u'id': 971, u'name': u'Chelsea LFC'},
 u'timestamp': u'00:11:35.842',
 u'type': {u'id': 30, u'name': u'Pass'}}

## Pässe "zählen"

In [13]:
r = {}
players = []
for row in out:
    if "recipient" in row["pass"].keys():
        players.append(row["player"]["name"])
        players.append(row["pass"]["recipient"]["name"])
        from_player = row["player"]["name"]
        to_player = row["pass"]["recipient"]["name"]
        if from_player+","+to_player in r:
            r[from_player+","+to_player] = r[from_player+","+to_player] + 1
        else:
            r[from_player+","+to_player] = 1
#players

# Netzwerkdaten erzeugen

In [158]:
G  = nx.DiGraph(name="team1")
G2 = nx.DiGraph(name="team2")

for item in list(set(players)):
    item_clean = re.sub(r"[^A-Za-z0-9]+", '',item)
    if item in players1:
        G.add_node(item_clean, size=1,label=item+" (" + players1[item] + ")", position=players1[item])
    elif item in players2:
        G2.add_node(item_clean, size=1,label=item+" (" + players2[item] + ")", position=players2[item])
j = 0
i = 0
for item,v in r.items():
    source = re.sub(r"[^A-Za-z0-9]+", '',item.split(",")[0])
    target = re.sub(r"[^A-Za-z0-9]+", '',item.split(",")[1])
    if item.split(",")[0] in players1:
        G.add_edge(source,target,weight=v) 
    elif item.split(",")[1] in players2:
        G2.add_edge(source,target,weight=v)
    else:
        print(item)

    
nx.write_gexf(G,"Team1.gexf")
nx.write_gexf(G2,"Team2.gexf")